In [ ]:
import sys
!{sys.executable} -m pip install PyAthena
from pyathena import connect 
import pandas as pd
import sagemaker
import boto3

In [ ]:
#TODO: Change the bucket name and replace it with your athena federation workshop bucket name , already created for you. We will use this s3 bucket for model output and training data
s3 = boto3.client('s3')
bucket = 'athena-federation-workshop-***********'
output_location = 's3://' + bucket + '/athena-ml/'
connection = connect(s3_staging_dir=output_location, region_name='us-east-1', work_group='AmazonAthenaPreviewFunctionality')

In [ ]:
## Select the entire dataset and save it to a variable to be used later to fit the model.
results = pd.read_sql('SELECT o_orderdate, count(*) as number FROM "lambda:mysql".sales.orders group by o_orderdate', connection)

In [ ]:
## Lets see the data we are working with
results

In [ ]:
from sagemaker import RandomCutForest

prefix = 'athena-ml/anomalydetection'
execution_role = sagemaker.get_execution_role()
session = sagemaker.Session()

# specify general training job information
rcf = RandomCutForest(role=execution_role,
                      train_instance_count=1,
                      train_instance_type='ml.c4.2xlarge',
                      data_location='s3://{}/{}/'.format(bucket, prefix),
                      output_path='s3://{}/{}/output'.format(bucket, prefix),
                      num_samples_per_tree=512,
                      num_trees=50)

# Run the training job using the results we got from the Athena query earlier
rcf.fit(rcf.record_set(results.number.values.reshape(-1,1)))

print('Training job name: {}'.format(rcf.latest_training_job.job_name))

rcf_inference = rcf.deploy(
    initial_instance_count=1,
    instance_type='ml.c4.2xlarge',
)

print('\nEndpoint name (used by Athena): {}'.format(rcf_inference.endpoint))